# RAMP - Urban Analytics: Microsim User Guide

## Introduction

This microsimulation models how a pathogen spreads through a population, taking into account where people live and the places they visit (e.g. retail, work, school). In this version of the model, we simulate COVID-19/SARS-CoV-2 transmission in England, but the model could be repurposed for other infectious diseases and/or geographical areas. Potential uses for the model include simulating the effect of starting and stopping different interventions (e.g. school closures) and predicting the impact of asmptomatic transmission.

This notebook is a work in progress focussing on currently implemented features. Please check back later for updated versions of the code and documentation.

## Overview of the model's components

This section shows how the different components (classes, functions, input data etc) fit together to simulate how danger scores for individuals and locations change over time.

### Input data

#### Individuals and households

SPENSER (Synthetic Population Estimation and Scenario Projection Model) is used to create population estimates at the household level for England (https://lida.leeds.ac.uk/research-projects/spenser-synthetic-population-estimation-and-scenario-projection-model/
). In other words, it generates a set of representative individuals (with a given age, sex and ethnicity) assigned to households. A household is one ore more individuals assumed to be living in the same 'home' (a physical space such as a house or flat).  

#### Health status

Individuals are given one of four possible health/disease states:
 - Susceptible (never had disease or no longer immune)
 - Exposed (individual has been infected but not (yet) infectious; incubation period)
 - Infectious (can infect others)
 - Recovered/Removed (individual has immunity or has died)

#### Activities

Individuals are assigned a job ('work', high level category for now), a primary or secondary school and/or shopping behaviour ('retail'). Each of these activities is assigned a duration (fraction of day the individual spends doing this) and one or more possible locations (e.g. individual has 1 office but may visit multiple shops). A 'flow' variable gives the probability of an individual going to each location (fractions which sum to 1 per activity).

#### Travel mode and time 

to be added

#### 'Danger' scores

When 'infectious' individuals visit locations, they increase the 'danger' score for that location. This translates into an increased likelihood ('risk') that another 'susceptible' individual travelling to the same location will become 'exposed'. 

The danger score assigned to each location is calculated as

$danger_l = \sum_{i=1}^{nr-infectious} ( duration_{il} * flow_{il} * danger$_$multiplier )$

The risk that individuals get from a location is calculatd as

$risk_i = \sum_{l=1}^{nr-locations} ( duration_{il} * flow_{il} * danger_l * risk$_$multiplier )$

Danger_multiplier and risk_multiplier are user-defined weights (default set to 1).

### Classes and functions

When running the code, a <b>Microsim</b> object is created which holds information about the individuals in the population (demographics, health and their activities, including where they go and for how long). The component activity_location points to an <b>ActivityLocation</b> object which holds information about the locations (mainly an identifier, flows and danger score). The ActivityLocation object references to an other class, <b>ColumnNames</b>, to ensure consistent column names.

After the initial Microsim object is created, it can then be updated through a series of steps (days). At each step, the danger scores assigned to all venues are updated based on how many infectious individuals have visited. Next, each individual gets an updated risk based on where they have been, and their disease status is updated accordingly.

#### Microsim functions
![title](microsim_defs.jpg)

#### ActivityLocation functions
![title](actloc_defs.jpg)

#### Resulting objects
![title](Objs.jpg)

## Example with dummy data

This section follow an example dummy population of ?17? individuals.

## Assumptions

This section discusses assumptions, simplifications etc